In [9]:
import pandas as pd
import numpy as np
import os

In [10]:
class TreeNode:
    def __init__(self, level, V, W, taken):
        self.level = level # niveau dans l’arbre de recherche >=0
        self.V = V # valeur de la solution courante
        self.W = W # poids de la solution courante
        self.taken = taken # liste des index d’objets de la solution courante
    def __str__(self):
        return str((self.level, self.V, self.W, self.taken))

In [11]:
class KPBB:
    def __init__(self, cap, values, weights): # Initialisation des données
        self.capacity = cap
        self.values = values
        self.weights = weights
        self.unordered = [((v, w), i) for i, (v, w) in  enumerate(zip(self.values, self.weights))]
        self.ordered = sorted([((v, w), i) for i, (v, w) in  enumerate(zip(self.values, self.weights))] , key = lambda tup: float(tup[0][0])/tup[0][1], reverse = True)
    
    def initialSolution(self): # calcul de la solution initiale
        E=0
        capInit=0
        sacInit=[]
        
        for i in range(len(self.weights)):
            if capInit+self.weights[i]>self.capacity:
                i=i+1
            else: 
                E=E+self.values[i]
                sacInit.append(i+1)
                capInit=capInit+self.weights[i]
                #i=i+1
        node=TreeNode(0,E,capInit,sacInit)                 
        return E,sacInit,capInit,node
    def complete(self,sac):
        complete=True
        #print(sac)
        for i in range(len(sac)):
            if sac[i][1]!=1:
                complete=False
                break
        return complete
    def nodeEvaluation(self,level,IndexesP): # fonction d’évaluation d’un noeud
        ENode=0
        capNode=0
        sacNode=[]
        for el in range(len(IndexesP)):
            if IndexesP[el]==1:
                sacNode.append((el+1,1))
        #print('sacNode:',sacNode)
        if level==0:
            for i in range(len(self.ordered)):
                if capNode+self.ordered[i][0][1]>self.capacity:
                    remaincap=self.capacity-capNode
                    coef=remaincap/self.ordered[i][0][1]
                    if coef==0 :
                        break
                        
                    capNode=self.capacity
                    sacNode.append((self.ordered[i][1]+1,coef))
                    ENode=ENode+(self.ordered[i][0][0]*coef)
                else: 
                    ENode=ENode+self.ordered[i][0][0]
                    sacNode.append((self.ordered[i][1]+1,1))
                    capNode=capNode+self.ordered[i][0][1]
                    i=i+1

            node=TreeNode(level,ENode,capNode,sacNode)
        else:
            for idx in range(len(IndexesP)):
                if IndexesP[idx]==1:
                    capNode=capNode+self.weights[idx]
                    ENode=ENode+self.values[idx]
            #print('capNode:',capNode)
            #print('ENode:',ENode)
            for i in range(len(self.ordered)):
                #print('self.ordered[i]:',self.ordered[i])
                if IndexesP[self.ordered[i][1]]==0:
                    if capNode+self.ordered[i][0][1]>self.capacity:
                        remaincap=self.capacity-capNode
                        coef=remaincap/self.ordered[i][0][1]
                        if coef==0 :
                            break
                        capNode=self.capacity
                        sacNode.append((self.ordered[i][1]+1,coef))
                        ENode=ENode+(self.ordered[i][0][0]*coef)
                    else: 
                        ENode=ENode+self.ordered[i][0][0]
                        sacNode.append((self.ordered[i][1]+1,1))
                        capNode=capNode+self.ordered[i][0][1]
                        i=i+1
            node=TreeNode(level,ENode,capNode,sacNode)   
        
        return node,self.complete(node.taken)
    def recursolveP(self,level,node,Arbre,LB,Index):
        while level < len(node.taken):
            level=level+1
            IndexR=Index
            IndexR[level-1]=1
            nodeR,completeR=self.nodeEvaluation(level, IndexR)
            if (completeR==True and nodeR.V>max(LB)):
                LB.append(nodeR.V)
            Arbre.append(nodeR)
            self.recursolve(level,nodeR,Arbre,LB,IndexR)
            IndexL=Index
            IndexL[level-1]=-1
            nodeL,completeL=self.nodeEvaluation(level, IndexL)
            if (completeL==True and nodeL.V>max(LB)):
                LB.append(nodeL.V)
            self.recursolve(level,nodeL,Arbre,LB,IndexL)
            Arbre.append(nodeL)
            
        return Arbre,LB
    def recursolveD(self,level,node,Arbre,LB,Index,SolSac):
        #while level < len(node.taken):
                level=level+1
                IndexR=Index
                IndexR[level-1]=1
                #print('Index partie R',IndexR)
                cap=0
                for el in range(len(IndexR)):
                    if IndexR[el]==1:
                        cap=cap+self.weights[el]
                if cap<=self.capacity:                   
                    nodeR,completeR=self.nodeEvaluation(level, IndexR)
                    Arbre.append(nodeR)
                    #print('nodeR',nodeR)
                    if (completeR==True and nodeR.V>max(LB)):
                        LB.append(nodeR.V)
                        SolSac.append(nodeR.taken)
                    if nodeR.V>max(LB):
                        self.recursolveD(level,nodeR,Arbre,LB,IndexR,SolSac)

                IndexL=Index
                IndexL[level-1]=-1
                IndexL[level:]=0
                #print('Index partie Left',IndexL)
                
                cap=0
                for el in range(len(IndexL)):
                    if IndexL[el]==1:
                        cap=cap+self.weights[el]
                #print('IndexL',IndexL)
                #print('level',level)
                if cap<=self.capacity:                   
                    nodeL,completeL=self.nodeEvaluation(level, IndexL)
                    Arbre.append(nodeL)
                    #print('nodeL',nodeL)
                    if (completeL==True and nodeL.V>max(LB)):
                        LB.append(nodeL.V)
                        SolSac.append(nodeL.taken)
                    if nodeL.V>max(LB):
                        self.recursolveD(level,nodeL,Arbre,LB,IndexL,SolSac)
                        
            
                return Arbre,LB
    def solve(self): # algorithme de résolution
        LB=[]
        SolSac=[]
        Arbre=[]
        EInit,sacInit,capInit,nodeIB=self.initialSolution()
        LB.append(EInit)
        SolSac.append(sacInit)
        IndexI=np.array([0 for i in range(len(self.weights))])
        nodeR,complete=self.nodeEvaluation(0, IndexI)
        if (complete==True and nodeR.V>max(LB)):
                LB.append(nodeR.V)
                SolSac.append(nodeR.taken)
        Arbre.append(nodeR)
        Index=np.array([0 for i in range(len(self.weights))])
        self.recursolveD(0,nodeR,Arbre,LB,Index,SolSac)
        #print(SolSac)
            
        
        
        
        return LB,Arbre,max(LB),SolSac[LB.index(max(LB))]

In [12]:
# Données du cours
cap=10
values=[40,50,100,95,30]
weights=[2,3,1,5,3]
kbb=KPBB(cap,values,weights)

In [13]:
print(kbb.capacity)

10


In [14]:
EInit,sacInit,capInit,nodeIB=kbb.initialSolution()

In [15]:
print(EInit)
print(sacInit)
print(capInit)
print(nodeIB)

220
[1, 2, 3, 5]
9
(0, 220, 9, [1, 2, 3, 5])


In [29]:
%timeit LB,Arbre,Solution,SacSol=kbb.solve()

271 µs ± 23.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [30]:
LB,Arbre,Solution,SacSol=kbb.solve()

In [31]:
LB

[220, 235, 245]

In [32]:
Solution

245

In [33]:
SacSol

[(2, 1), (3, 1), (4, 1)]

In [40]:
cap=269
values=[55,10,47,5,4,50,8,61,85,87]
print(len(values))
weights=[95,4,60,32,23,72,80,62,65,46]
print(len(weights))
kbb=KPBB(cap,values,weights)

10
10


In [41]:
%timeit LB,Arbre,Solution,SacSol=kbb.solve()

2.56 ms ± 205 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [42]:
LB,Arbre,Solution,SacSol=kbb.solve()

In [43]:
LB

[208, 260, 274, 288, 295]

In [44]:
Solution

295

In [45]:
SacSol

[(2, 1), (3, 1), (4, 1), (10, 1), (9, 1), (8, 1)]

In [46]:
cap=878
values=[44,46,90,72,91,40,75,35,8,54,78,40,77,15,61,17,75,29,75,63]
print(len(values))
weights=[92,4,43,83,84,68,92,82,6,44,32,18,56,83,25,96,70,48,14,58]
print(len(weights))
kbb=KPBB(cap,values,weights)

20
20


In [47]:
%timeit LB,Arbre,Solution,SacSol=kbb.solve()

3.31 ms ± 234 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [48]:
LB,Arbre,Solution,SacSol=kbb.solve()

In [49]:
LB

[930, 990, 1024]

In [50]:
Solution

1024

In [51]:
SacSol

[(1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (15, 1),
 (17, 1),
 (19, 1),
 (20, 1)]

In [52]:
def chargement_fichier(chemin):
    data = pd.read_csv(chemin)
    cap=int(data.columns[0].split()[1])
    data = pd.read_csv(chemin, header=None, names=(['v','w']),sep=" ")
    v=data.v.values
    w=data.w.values
    return cap,v,w

In [73]:
cap,values,weights=chargement_fichier(r"C:\\Users\\surfaC\\Desktop\\Master SID\\Optimisation Combinatoire\\TP\\TP4\\dataKnapsack\\large_scale\\knapPI_1_100_1000_1")

In [74]:
weights

array([995, 485, 326, 248, 421, 322, 795,  43, 845, 955, 252,   9, 901,
       122,  94, 738, 574, 715, 882, 367, 984, 299, 433, 682,  72, 874,
       138, 856, 145, 995, 529, 199, 277,  97, 719, 242, 107, 122,  70,
        98, 600, 645, 267, 972, 895, 213, 748, 487, 923,  29, 674, 540,
       554, 467,  46, 710, 553, 191, 724, 730, 988,  90, 340, 549, 196,
       865, 678, 570, 936, 722, 651, 123, 431, 508, 585, 853, 642, 992,
       725, 286, 812, 859, 663,  88, 179, 187, 619, 261, 846, 192, 261,
       514, 886, 530, 849, 294, 799, 391, 330, 298, 790], dtype=int64)

In [75]:
kbb=KPBB(cap,values,weights)

In [76]:
%timeit LB,Arbre,Solution,SacSol=kbb.solve()

1.33 s ± 57.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [77]:
LB,Arbre,Solution,SacSol=kbb.solve()

In [78]:
LB

[100,
 2515,
 3170,
 3272,
 3480,
 3503,
 3746,
 3790,
 4011,
 4090,
 4113,
 4321,
 4505,
 4905,
 5146,
 5603,
 5811,
 5913,
 6013,
 6120,
 6131,
 6187,
 6233,
 6343,
 6417,
 6432,
 6866,
 6889,
 7048,
 7082,
 7097,
 7108,
 7199,
 7210,
 7306,
 7551,
 7625,
 7782,
 7807,
 7835,
 7909,
 7935,
 7968,
 8109,
 8279,
 8436,
 8510,
 8556,
 8586,
 8745,
 8817,
 8940,
 8990,
 9147]

In [79]:
Solution

9147

In [80]:
SacSol

[(8, 1),
 (12, 1),
 (15, 1),
 (25, 1),
 (27, 1),
 (32, 1),
 (34, 1),
 (39, 1),
 (40, 1),
 (50, 1),
 (55, 1),
 (62, 1)]

In [85]:
cap,values,weights=chargement_fichier(r"C:\\Users\\surfaC\\Desktop\\Master SID\\Optimisation Combinatoire\\TP\\TP4\\dataKnapsack\\large_scale\\knapPI_3_200_1000_1")

In [86]:
weights

array([997, 485,  94, 326, 506, 248, 416, 421, 992, 322, 649, 795, 237,
        43, 457, 845, 815, 955, 446, 252, 422,   9, 791, 901, 359, 122,
       667,  94, 598, 738,   7, 574, 544, 715, 334, 882, 766, 367, 994,
       984, 893, 299, 633, 433, 131, 682, 428,  72, 700, 874, 617, 138,
       874, 856, 720, 145, 419, 995, 794, 529, 196, 199, 997, 277, 116,
        97, 908, 719, 539, 242, 707, 107, 569, 122, 537,  70, 931,  98,
       726, 600, 487, 645, 772, 267, 513, 972,  81, 895, 943, 213,  58,
       748, 303, 487, 764, 923, 536,  29, 724, 674, 789, 540, 479, 554,
       142, 467, 339,  46, 641, 710, 196, 553, 494, 191,  66, 724, 824,
       730, 208, 988, 711,  90, 800, 340, 314, 549, 289, 196, 401, 865,
       466, 678, 689, 570, 833, 936, 225, 722, 244, 651, 849, 123, 113,
       431, 379, 508, 361, 585,  65, 853, 486, 642, 686, 992, 286, 725,
       889, 286,  24, 812, 491, 859, 891, 663,  90,  88, 181, 179, 214,
       187,  17, 619, 472, 261, 418, 846, 419, 192, 356, 261, 68

In [87]:
kbb=KPBB(cap,values,weights)

In [88]:
%timeit LB,Arbre,Solution,SacSol=kbb.solve()

2min 39s ± 12.9 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [89]:
LB,Arbre,Solution,SacSol=kbb.solve()

In [90]:
LB

[200,
 1693,
 1696,
 1791,
 1797,
 1897,
 1991,
 2096,
 2097,
 2190,
 2197,
 2291,
 2292,
 2294,
 2295,
 2297,
 2383,
 2390,
 2394,
 2396,
 2397,
 2494,
 2497,
 2597,
 2694,
 2695,
 2696,
 2697]

In [91]:
Solution

2697

In [92]:
SacSol

[(3, 1),
 (14, 1),
 (22, 1),
 (26, 1),
 (31, 1),
 (48, 1),
 (66, 1),
 (76, 1),
 (171, 1),
 (159, 1),
 (98, 1),
 (108, 1),
 (91, 1),
 (149, 1),
 (115, 1),
 (166, 1),
 (122, 1)]